In [27]:
import tensorflow as tf
import tensorflow.lite as tflite
import numpy as np
from tensorflow import keras
from io import BytesIO
from urllib import request
from PIL import Image
from keras.applications.xception import preprocess_input

Load original model:

In [28]:
model = keras.models.load_model('./model_2024_hairstyle.keras')

Convert original model to lite version:

In [29]:

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('model_2024_hairstyle.tflite','wb') as f_out:
	f_out.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\super\AppData\Local\Temp\tmpdhrrvapg\assets


INFO:tensorflow:Assets written to: C:\Users\super\AppData\Local\Temp\tmpdhrrvapg\assets


Saved artifact at 'C:\Users\super\AppData\Local\Temp\tmpdhrrvapg'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 200, 200, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  2680332092240: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2680332094736: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2680332094928: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2680332095888: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2680332095504: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2680332096848: TensorSpec(shape=(), dtype=tf.resource, name=None)


Load Lite version model:

In [30]:
interpreter = tflite.Interpreter(model_path = "./model_2024_hairstyle.tflite")
interpreter.allocate_tensors()

In [ ]:
interpreter.get_output_details()

[{'name': 'StatefulPartitionedCall_1:0',
  'index': 13,
  'shape': array([1, 1]),
  'shape_signature': array([-1,  1]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [41]:
output_index = interpreter.get_output_details()[0]['index']
output_index

13

In [42]:
interpreter.get_input_details()

[{'name': 'serving_default_input_layer:0',
  'index': 0,
  'shape': array([  1, 200, 200,   3]),
  'shape_signature': array([ -1, 200, 200,   3]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [45]:
input_index = interpreter.get_input_details()[0]['index']
input_index

0

In [33]:
def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img
def prepare_input(x):
    return x / 255.0

In [34]:
img = download_image("https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg")

In [35]:
img = prepare_image(img, (200,200))

In [36]:
x = np.array(img, dtype='float32')
X = np.array([x])
X = prepare_input(X)


In [37]:
X[0, 0, 0, 0]

0.23921569

In [46]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()

preds = interpreter.get_tensor(output_index)

In [47]:
preds

array([[0.8934675]], dtype=float32)